# Crescent Engineering Contracting Strategy Dashboard

Interactive dashboard for tracking project milestones, schedules, and dependencies.

In [1]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import numpy as np
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load data from Excel
excel_file = 'Crescent - Engineering Contracting Strategy.xlsx'
df_sheet1 = pd.read_excel(excel_file, sheet_name='Sheet1')
df_sheet2 = pd.read_excel(excel_file, sheet_name='Sheet2')

# Clean the data
df = df_sheet1.copy()
df = df.rename(columns={'Milestone / Deliverable': 'Milestone', 'Dependencies/Notes': 'Notes'})

# Convert dates to datetime
df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
df['Finish'] = pd.to_datetime(df['Finish'], errors='coerce')

# Calculate duration in days
df['Duration'] = (df['Finish'] - df['Start']).dt.days
df['Duration'] = df['Duration'].fillna(0)

# For tasks without start date, use finish date minus 1 day
df.loc[df['Start'].isna(), 'Start'] = df.loc[df['Start'].isna(), 'Finish'] - pd.Timedelta(days=1)

# Calculate progress (assume today's date)
today = pd.Timestamp.now()
df['Progress'] = 0.0

# Calculate progress for each task
for idx, row in df.iterrows():
    if pd.notna(row['Start']) and pd.notna(row['Finish']):
        if today >= row['Finish']:
            df.at[idx, 'Progress'] = 100
        elif today <= row['Start']:
            df.at[idx, 'Progress'] = 0
        else:
            total_days = (row['Finish'] - row['Start']).days
            elapsed_days = (today - row['Start']).days
            df.at[idx, 'Progress'] = (elapsed_days / total_days * 100) if total_days > 0 else 0

# Add status based on progress
df['Status'] = df['Progress'].apply(lambda x: 'Completed' if x >= 100 else ('In Progress' if x > 0 else 'Not Started'))

# Add color coding
df['Color'] = df['Owner'].map({
    'AES': '#2E86AB',
    'PRE': '#A23B72',
    'AES ↔ PRE': '#F18F01'
})

print(f"Dashboard loaded: {len(df)} milestones tracked")
print(f"Date range: {df['Start'].min().strftime('%Y-%m-%d')} to {df['Finish'].max().strftime('%Y-%m-%d')}")

# Crescent Engineering Contracting Strategy - Unified Dashboard

In [3]:
# Create unified dashboard
from plotly.subplots import make_subplots

# Calculate statistics
total_tasks = len(df)
completed_tasks = len(df[df['Status'] == 'Completed'])
in_progress_tasks = len(df[df['Status'] == 'In Progress'])
not_started_tasks = len(df[df['Status'] == 'Not Started'])
overall_progress = df['Progress'].mean()

# Display header statistics as cards
display(HTML(f"""
<div style='display: flex; gap: 20px; margin: 20px 0; flex-wrap: wrap;'>
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                color: white; padding: 25px; border-radius: 15px; flex: 1; min-width: 200px;
                box-shadow: 0 10px 30px rgba(0,0,0,0.2);'>
        <div style='font-size: 14px; opacity: 0.9; margin-bottom: 10px;'>Total Milestones</div>
        <div style='font-size: 42px; font-weight: bold;'>{total_tasks}</div>
    </div>
    <div style='background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
                color: white; padding: 25px; border-radius: 15px; flex: 1; min-width: 200px;
                box-shadow: 0 10px 30px rgba(0,0,0,0.2);'>
        <div style='font-size: 14px; opacity: 0.9; margin-bottom: 10px;'>In Progress</div>
        <div style='font-size: 42px; font-weight: bold;'>{in_progress_tasks}</div>
    </div>
    <div style='background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); 
                color: white; padding: 25px; border-radius: 15px; flex: 1; min-width: 200px;
                box-shadow: 0 10px 30px rgba(0,0,0,0.2);'>
        <div style='font-size: 14px; opacity: 0.9; margin-bottom: 10px;'>Not Started</div>
        <div style='font-size: 42px; font-weight: bold;'>{not_started_tasks}</div>
    </div>
    <div style='background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%); 
                color: white; padding: 25px; border-radius: 15px; flex: 1; min-width: 200px;
                box-shadow: 0 10px 30px rgba(0,0,0,0.2);'>
        <div style='font-size: 14px; opacity: 0.9; margin-bottom: 10px;'>Overall Progress</div>
        <div style='font-size: 42px; font-weight: bold;'>{overall_progress:.1f}%</div>
    </div>
</div>
"""))

# Create comprehensive dashboard with subplots
fig = make_subplots(
    rows=3, cols=2,
    row_heights=[0.5, 0.25, 0.25],
    column_widths=[0.6, 0.4],
    specs=[
        [{'type': 'xy', 'rowspan': 1, 'colspan': 2}, None],
        [{'type': 'xy'}, {'type': 'pie'}],
        [{'type': 'xy'}, {'type': 'bar'}]
    ],
    subplot_titles=(
        'Project Timeline (Gantt Chart)',
        'Progress by Milestone',
        'Task Status Distribution',
        'Tasks by Owner',
        ''
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Row 1: Gantt Chart (spans full width)
for idx, row in df.iterrows():
    # Background bar
    fig.add_trace(go.Bar(
        y=[row['Milestone']],
        x=[row['Duration']],
        base=row['Start'],
        orientation='h',
        marker=dict(color=row['Color'], opacity=0.3, line=dict(color=row['Color'], width=2)),
        name=row['Owner'],
        showlegend=False,
        hovertemplate=(
            f"<b>{row['Milestone']}</b><br>"
            f"Owner: {row['Owner']}<br>"
            f"Start: {row['Start'].strftime('%Y-%m-%d')}<br>"
            f"Finish: {row['Finish'].strftime('%Y-%m-%d')}<br>"
            f"Duration: {row['Duration']} days<br>"
            f"Progress: {row['Progress']:.1f}%<br>"
            f"Status: {row['Status']}<extra></extra>"
        )
    ), row=1, col=1)
    
    # Progress bar
    if row['Progress'] > 0:
        progress_duration = row['Duration'] * (row['Progress'] / 100)
        fig.add_trace(go.Bar(
            y=[row['Milestone']],
            x=[progress_duration],
            base=row['Start'],
            orientation='h',
            marker=dict(color=row['Color'], opacity=0.9),
            showlegend=False,
            hovertemplate=f"Progress: {row['Progress']:.1f}%<extra></extra>"
        ), row=1, col=1)

# Add today line to Gantt
fig.add_vline(
    x=today.timestamp() * 1000,
    line_dash="dash",
    line_color="red",
    annotation_text="Today",
    annotation_position="top",
    row=1, col=1
)

# Row 2, Col 1: Progress by Milestone
fig.add_trace(go.Bar(
    y=df['Milestone'],
    x=df['Progress'],
    orientation='h',
    marker=dict(
        color=df['Progress'],
        colorscale=[[0, '#e74c3c'], [0.5, '#f39c12'], [1, '#27ae60']],
        showscale=False
    ),
    text=df['Progress'].round(1).astype(str) + '%',
    textposition='inside',
    showlegend=False,
    hovertemplate='<b>%{y}</b><br>Progress: %{x:.1f}%<extra></extra>'
), row=2, col=1)

# Row 2, Col 2: Status pie chart
status_counts = df['Status'].value_counts()
fig.add_trace(go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['#43e97b', '#f093fb', '#4facfe']),
    hole=0.4,
    showlegend=True
), row=2, col=2)

# Row 3, Col 2: Tasks by owner
owner_counts = df['Owner'].value_counts()
fig.add_trace(go.Bar(
    x=owner_counts.index,
    y=owner_counts.values,
    marker=dict(color=['#2E86AB', '#A23B72', '#F18F01']),
    text=owner_counts.values,
    textposition='auto',
    showlegend=False
), row=3, col=2)

# Update layout
fig.update_xaxes(type='date', gridcolor='#ecf0f1', showgrid=True, row=1, col=1)
fig.update_yaxes(autorange='reversed', gridcolor='#ecf0f1', row=1, col=1)
fig.update_yaxes(autorange='reversed', gridcolor='#ecf0f1', row=2, col=1)
fig.update_xaxes(range=[0, 100], gridcolor='#ecf0f1', showgrid=True, title='Progress (%)', row=2, col=1)
fig.update_xaxes(title='Owner', row=3, col=2)
fig.update_yaxes(title='Count', row=3, col=2)

fig.update_layout(
    title={
        'text': 'Crescent Engineering Contracting Strategy Dashboard',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2c3e50'}
    },
    height=1400,
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True,
    barmode='overlay',
    font=dict(family='Arial, sans-serif', size=11)
)

fig.show()

# Show upcoming milestones
next_30_days = today + pd.Timedelta(days=30)
upcoming = df[(df['Finish'] >= today) & (df['Finish'] <= next_30_days)].sort_values('Finish')

if len(upcoming) > 0:
    display(HTML("<h3 style='color: #3498db; margin-top: 30px;'>📅 Upcoming Milestones (Next 30 Days)</h3>"))
    
    for idx, row in upcoming.iterrows():
        days_until = (row['Finish'] - today).days
        urgency_color = '#e74c3c' if days_until <= 7 else '#f39c12' if days_until <= 14 else '#3498db'
        
        display(HTML(f"""
        <div style='background: white; border-left: 5px solid {urgency_color};
                    padding: 15px; margin: 10px 0; border-radius: 8px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);'>
            <div style='display: flex; justify-content: space-between; align-items: center;'>
                <div style='font-size: 16px; font-weight: bold; color: #2c3e50;'>{row['Milestone']}</div>
                <div style='background: {urgency_color}; color: white; padding: 5px 15px; 
                            border-radius: 20px; font-weight: bold;'>{days_until} days</div>
            </div>
            <div style='margin-top: 10px; color: #7f8c8d;'>
                <strong>Owner:</strong> {row['Owner']} | <strong>Due:</strong> {row['Finish'].strftime('%Y-%m-%d')} | 
                <strong>Progress:</strong> {row['Progress']:.1f}%
            </div>
        </div>
        """))

# Show detailed table
display(HTML("<h3 style='color: #2c3e50; margin-top: 30px;'>📋 Detailed Milestone Information</h3>"))
display_df = df[['Milestone', 'Owner', 'Start', 'Finish', 'Duration', 'Progress', 'Status', 'Notes']].copy()
display_df['Start'] = display_df['Start'].dt.strftime('%Y-%m-%d')
display_df['Finish'] = display_df['Finish'].dt.strftime('%Y-%m-%d')
display_df['Progress'] = display_df['Progress'].round(1).astype(str) + '%'

styled_df = display_df.style.set_properties(**{
    'background-color': 'white', 'color': '#2c3e50', 'border': '1px solid #ecf0f1',
    'padding': '10px', 'text-align': 'left'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#34495e'), ('color', 'white'), ('font-weight', 'bold'), ('padding', '12px')]},
    {'selector': 'tr:hover', 'props': [('background-color', '#ecf0f1')]}
])

display(styled_df)

,Milestone,Owner,Start,Finish,Duration,Progress,Status,Notes
0,Project engineering document handoff,AES,2025-10-03,2025-10-06,3.000000,0.0%,Not Started,What can be used from Redonda if anything?
1,AES provide external folder with EPCA exhibits,AES ↔ PRE,2025-10-09,2025-10-10,0.000000,0.0%,Not Started,Populate external drive with necessary exhibits; requires coordination with SMEs.
2,Internal layout modeling (510ac & 550ac),AES,2025-10-10,2025-10-17,7.000000,0.0%,Not Started,Modeling required to compare 510ac (2) GSU arrangement to 550ac (3) GSU arrangement. Output feeds layout decision.
3,layout decision,AES,2025-10-16,2025-10-17,0.000000,0.0%,Not Started,Dependent on internal modeling results; selection needed before PRE can lock 10% redesign and pricing.
4,"Layout Redesign (PRE 10%, schedule, and binding pricing)",PRE,2025-10-10,2025-11-21,42.000000,0.0%,Not Started,"PRE to advance 10% layout + preliminary schedule + binding pricing. Must incorporate AES final layout choice, exhibit package, and clarification/ACE items. Assumes landowner feedback and geotech baseline are stable."
5,Updated PPA offer submission,AES,2025-10-19,2025-10-20,0.000000,0.0%,Not Started,Requires interim pricing basis for Google (510 vs. 550 option). AES submits updated offer in parallel with layout work; outcome informs contracting strategy.
6,PPA decision,AES,2025-10-30,2025-10-31,0.000000,0.0%,Not Started,Google approval of updated offer; critical gating item. EPCA path only advances if PPA approved.
7,EPCA negotiations,AES ↔ PRE,2025-11-21,2025-12-15,24.000000,0.0%,Not Started,nan
8,Earliest possible EPCA execution,AES ↔ PRE,2025-12-14,2025-12-15,0.000000,0.0%,Not Started,"Requires: (i) PRE binding pricing and allowances log (Nov 21), (ii) true-up of any open ACE/tech spec deviations, (iii) completed exhibit set. Target aligns with PRE’s request to close EPCA by year-end."


In [7]:
# Create progress bar chart
fig_progress = go.Figure()

fig_progress.add_trace(go.Bar(
    y=df['Milestone'],
    x=df['Progress'],
    orientation='h',
    marker=dict(
        color=df['Progress'],
        colorscale=[
            [0, '#e74c3c'],
            [0.5, '#f39c12'],
            [1, '#27ae60']
        ],
        showscale=True,
        colorbar=dict(title='Progress %')
    ),
    text=df['Progress'].round(1).astype(str) + '%',
    textposition='inside',
    hovertemplate='<b>%{y}</b><br>Progress: %{x:.1f}%<extra></extra>'
))

fig_progress.update_layout(
    title={
        'text': 'Progress by Milestone',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    xaxis=dict(
        title='Progress (%)',
        range=[0, 100],
        gridcolor='#ecf0f1',
        showgrid=True
    ),
    yaxis=dict(
        autorange='reversed',
        gridcolor='#ecf0f1'
    ),
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=12)
)

fig_progress.show()

In [11]:
# Option to export filtered data
print("To export current data to CSV, run:")
print("df.to_csv('crescent_dashboard_export.csv', index=False)")
print("\nTo export to Excel, run:")
print("df.to_excel('crescent_dashboard_export.xlsx', index=False)")

To export current data to CSV, run:
df.to_csv('crescent_dashboard_export.csv', index=False)

To export to Excel, run:
df.to_excel('crescent_dashboard_export.xlsx', index=False)
